In [ ]:
#| default_exp logger

## Logger Class

In [ ]:
#| hide

import sys
from pathlib import Path

sys.path.append(str(Path('.').absolute().parent))

In [ ]:
#| exporti

import watchtower, logging, json

from lakeinterface.config import ConfigManager

In [ ]:
#| exporti

def _boto_filter(record):
    # Filter log messages from botocore and its dependency, urllib3, in watchtower handler for CloudWatch.
    # This is required to avoid an infinite loop when shutting down.
    if record.name.startswith("botocore"):
        return False
    if record.name.startswith("urllib3"):
        return False
    return True

In [ ]:
#| export


class Logger(object):
    """
    A class to wrap standard python logger but adding a handler to write to AWS CloudWatch
    ...

    Attributes
    ----------
    logger: a standard python logger
    
    Methods
    -------
    __init__(logger_name, handlers_config, profile_name='default'):
        Initializes the logger and add any handlers defined in the handlers_confif
    
    add_stream_handler(handler_def):
        Add a handler for logging to console
    
    add_file_handler(handler_def):
        Add a handler for logging to file
    
    add_cloudwatch_handler(handler):
        Add a handler for logging to AWS Cloud Watch

    debug(msg) / info(msg) / warning(msg) / error(msg):
        Write logs of the relevant level
        
    clear_all_handlers():
        Remove all handlers from logger
    """
    
    def __init__(self, logger_name, handlers_config, profile_name='default'):
        self.logger = logging.getLogger(logger_name)
        
        for h in handlers_config:
            match h['handler_type']:
                case 'stream':
                    self.add_stream_handler(h)
                case 'file':
                    self.add_file_handler(h)
                case 'cloudwatch':
                    self.add_cloudwatch_handler(h)
                    
        self.logger.setLevel(min(h.level for h in self.logger.handlers))

    def add_stream_handler(self, handler_def):
        # check for existing stream handler
        if any(type(h)==logging.StreamHandler for h in self.logger.handlers):
            print('Console Logging Handler already attached')
            return
        
        print('adding stream', handler_def)
        c_handler = logging.StreamHandler()
        c_handler.setLevel(handler_def.get('level'))
        c_format = logging.Formatter(handler_def.get('format'))
        c_handler.setFormatter(c_format)
        self.logger.addHandler(c_handler)
        return 'stream'
        
    def add_file_handler(self, handler_def):
        # check for existing file handler
        if any(type(h)==logging.FileHandler for h in self.logger.handlers):
            print('File Logging Handler already attached')
            return
        
        print('adding file', handler_def)
        f_handler = logging.FileHandler(handler_def.get('log_file_path'))
        f_handler.setLevel(handler_def.get('level'))
        f_format = logging.Formatter(handler_def.get('format'))
        f_handler.setFormatter(f_format)
        self.logger.addHandler(f_handler)
        return 'file'
        
    def add_cloudwatch_handler(self, handler_def):
        if any(type(h)==watchtower.CloudWatchLogHandler for h in self.logger.handlers):
            print('Cloudwatch Logging Handler already attached')
            return
            
        print('adding cloudwatch', handler_def)
        wtower_handler = watchtower.CloudWatchLogHandler(
            log_group_name=handler_def.get('log_group_name'),
            log_stream_name=handler_def.get('log_stream_name'),
            send_interval=10,
            create_log_group=False,
            boto3_profile_name=handler_def.get('aws_profile_name')
        )
        wtower_handler.setLevel(handler_def.get('level'))
        logger.addFilter(_boto_filter)
        
        w_format = logging.Formatter(handler_def.get('format'))
        wtower_handler.setFormatter(w_format)
        self.logger.addHandler(wtower_handler)
        return 'cloudwatch'
    
    def debug(self, msg):
        self.logger.debug(msg)
    
    def info(self, msg):
        self.logger.info(msg)
    
    def warning(self, msg):
        self.logger.warning(msg)
    
    def error(self, msg):
        self.logger.error(msg)
    
    def clear_all_handlers(self):
        while self.logger.hasHandlers():
            self.logger.removeHandler(self.logger.handlers[0])


### Example config

All handlers listed in the config will be attached to logger unless there is already one attached

In [ ]:
handlers_config = [
    {
        'handler_type': 'stream', 
        'level': logging.INFO, 
        'format': '%(name)s - %(levelname)s - %(message)s'
    },
    {
        'handler_type': 'file', 
        'log_file_path': 'bankdata.log', 
        'level': logging.ERROR, 
        'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    },
    {
        'handler_type': 'cloudwatch', 
        'log_group_name': 'machinesp/test', 
        'log_stream_name': 'lake_tester', 
        'level': logging.DEBUG, 
        'aws_profile_name': 'personal',
        'format': '%(levelname)s - %(message)s'
    }
]

In [ ]:
bankdata_logger = Logger('bankdata', handlers_config)

adding stream {'handler_type': 'stream', 'level': 20, 'format': '%(name)s - %(levelname)s - %(message)s'}
adding file {'handler_type': 'file', 'log_file_path': 'bankdata.log', 'level': 40, 'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'}
adding cloudwatch {'handler_type': 'cloudwatch', 'log_group_name': 'machinesp/test', 'log_stream_name': 'lake_tester', 'level': 10, 'aws_profile_name': 'personal', 'format': '%(levelname)s - %(message)s'}


In [ ]:
bankdata_logger.info(json.dumps({'foo':'bar'}))

bankdata - INFO - {"foo": "bar"}


### Clear all attached handlers

In [ ]:
bankdata_logger.clear_all_handlers()
bankdata_logger.logger.handlers


[]